In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as  plt
import os, glob

import openmeteo_requests

import requests_cache
from retry_requests import retry

## Import the City Name, Latitude and Longitude values from the CSV.

In [2]:
saudi_cities = pd.read_csv(r"./Saudi Cities.csv")
saudi_cities.head()

,SCALERANK,NATSCALE,LABELRANK,FEATURECLA,NAME,NAMEPAR,NAMEALT,NAMEASCII,ADM0CAP,CAPIN,...,FCLASS_TR,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,FCLASS_TLC
0,10,1,5,Admin-1 capital,An Nabk,Qurayyat,,An Nabk,0,,...,,,,,,,,,,
1,9,5,5,Admin-1 capital,Sakakah,,,Sakakah,0,,...,,,,,,,,,,
2,8,10,5,Populated place,Yanbu al Bahr,,,Yanbu al Bahr,0,,...,,,,,,,,,,
3,8,10,5,Populated place,Dawmat al Jandal,,,Dawmat al Jandal,0,,...,,,,,,,,,,
4,8,10,5,Populated place,Qal at Bishah,,,Qal at Bishah,0,,...,,,,,,,,,,


In [3]:
saudi_cities.columns

Index(['SCALERANK', 'NATSCALE', 'LABELRANK', 'FEATURECLA', 'NAME', 'NAMEPAR',
       'NAMEALT', 'NAMEASCII', 'ADM0CAP', 'CAPIN',
       ...
       'FCLASS_TR', 'FCLASS_ID', 'FCLASS_PL', 'FCLASS_GR', 'FCLASS_IT',
       'FCLASS_NL', 'FCLASS_SE', 'FCLASS_BD', 'FCLASS_UA', 'FCLASS_TLC'],
      dtype='object', length=137)

In [4]:
select_cols = ["NAME", "LATITUDE", "LONGITUDE"]
saudi_cities_select = saudi_cities[select_cols]
saudi_cities_select.head()

,NAME,LATITUDE,LONGITUDE
0,An Nabk,31.333302,37.333297
1,Sakakah,29.999997,40.133305
2,Yanbu al Bahr,24.094277,38.049295
3,Dawmat al Jandal,29.815298,39.866393
4,Qal at Bishah,20.008687,42.598681


In [5]:
saudi_cities_select.shape

(31, 3)

In [6]:
saudi_cities['LATITUDE'][0]

np.float64(31.333302)

## API

In [7]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [8]:
def api_call(lat, long, url, start_date, end_date):
    params = {
        "latitude": lat,
        "longitude": long,
        "start_date": start_date,
        "end_date": end_date,
        "daily": ["temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "daylight_duration", "sunshine_duration"],
        "timezone": "Europe/Moscow"
    }
    response = openmeteo.weather_api(url, params=params)
    return response

In [15]:
responses = []
for i in range(saudi_cities_select.shape[0]):
    response = api_call(saudi_cities_select['LATITUDE'][i], saudi_cities_select['LONGITUDE'][i], "https://archive-api.open-meteo.com/v1/archive",start_date="2015-06-06", end_date="2015-06-06")
    responses.append(response)

# Looping through the list of responses and adding each cities response to the list of "response_dfs". 
response_dfs = []
for i in range(len(responses)):
	# daily response
	response = responses[i][0]
	daily = response.Daily()

	# daily data as requested in the "daily" parameters.
	daily = response.Daily()
	daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
	daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
	daily_temperature_2m_mean = daily.Variables(2).ValuesAsNumpy()
	daily_daylight_duration = daily.Variables(3).ValuesAsNumpy()
	daily_sunshine_duration = daily.Variables(4).ValuesAsNumpy()

	# formatting columns.

	daily_data = {"date": pd.date_range(
		start = pd.to_datetime(daily.Time(), unit = "s", utc=True), 
		end= pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
		freq = pd.Timedelta(seconds=daily.Interval()),
		inclusive="left")}
	daily_data["city_name"] = saudi_cities_select['NAME'][i]
	daily_data['latitude'] = saudi_cities_select['LATITUDE'][i]
	daily_data['longitude'] = saudi_cities_select['LONGITUDE'][i]
	daily_data["temperature_2m_max"] = daily_temperature_2m_max
	daily_data["temperature_2m_min"] = daily_temperature_2m_min
	daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
	daily_data["daylight_duration"] = daily_daylight_duration
	daily_data["sunshine_duration"] = daily_sunshine_duration


	response_dfs.append(pd.DataFrame(data = daily_data))

In [21]:
weather_df = pd.concat(response_dfs, axis=0).reset_index(drop=True)
weather_df.head(10)

,date,city_name,latitude,longitude,temperature_2m_max,temperature_2m_min,temperature_2m_mean,daylight_duration,sunshine_duration
0,2015-06-05 21:00:00+00:00,An Nabk,31.333302,37.333297,32.271000,14.671000,24.377253,50779.707031,46800.000000
1,2015-06-05 21:00:00+00:00,Sakakah,29.999997,40.133305,33.789497,18.589500,27.291586,50397.832031,46374.296875
2,2015-06-05 21:00:00+00:00,Yanbu al Bahr,24.094277,38.049295,36.948002,28.098000,32.814663,48837.937500,45963.269531
3,2015-06-05 21:00:00+00:00,Dawmat al Jandal,29.815298,39.866393,35.449001,19.448999,28.496918,50358.339844,46404.597656
4,2015-06-05 21:00:00+00:00,Qal at Bishah,20.008687,42.598681,38.934498,19.634499,31.232420,47856.941406,43200.000000
5,2015-06-05 21:00:00+00:00,At Taif,21.262228,40.382279,35.099503,25.199499,29.976585,48154.492188,43796.003906
6,2015-06-05 21:00:00+00:00,Najran,17.506540,44.131559,36.256500,20.006500,30.135672,47293.738281,43200.000000
7,2015-06-05 21:00:00+00:00,Al Quwayiyah,24.073710,45.280636,42.072002,25.972000,34.507420,48837.937500,43200.000000
8,2015-06-05 21:00:00+00:00,Al Hillah,23.489456,46.756360,41.687000,27.837000,35.266167,48698.601562,43588.757812
9,2015-06-05 21:00:00+00:00,Al Mubarraz,25.429054,49.565905,43.573997,22.874001,33.771912,49174.664062,45348.429688


In [22]:
weather_df.to_csv(r"./Saudi Weather.csv")

In [4]:
# # Process first location. Add a for-loop for multiple locations or weather models
# response = responses[0]
# print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
# print(f"Elevation {response.Elevation()} m asl")
# print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
# print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# # Process daily data. The order of variables needs to be the same as requested.
# daily = response.Daily()
# daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
# daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
# daily_temperature_2m_mean = daily.Variables(2).ValuesAsNumpy()
# daily_daylight_duration = daily.Variables(3).ValuesAsNumpy()
# daily_sunshine_duration = daily.Variables(4).ValuesAsNumpy()

# daily_data = {"date": pd.date_range(
# 	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
# 	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
# 	freq = pd.Timedelta(seconds = daily.Interval()),
# 	inclusive = "left"
# )}
# daily_data["temperature_2m_max"] = daily_temperature_2m_max
# daily_data["temperature_2m_min"] = daily_temperature_2m_min
# daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
# daily_data["daylight_duration"] = daily_daylight_duration
# daily_data["sunshine_duration"] = daily_sunshine_duration

# daily_dataframe = pd.DataFrame(data = daily_data)
# print(daily_dataframe)

Coordinates 24.63971710205078°N 46.73447799682617°E
Elevation 607.0 m asl
Timezone b'Europe/Moscow' b'MSK'
Timezone difference to GMT+0 10800 s
                          date  temperature_2m_max  temperature_2m_min  \
0    2015-06-05 21:00:00+00:00           43.274502           25.624498   
1    2015-06-06 21:00:00+00:00           42.974503           23.924500   
2    2015-06-07 21:00:00+00:00           42.174500           27.724499   
3    2015-06-08 21:00:00+00:00           40.824501           27.024500   
4    2015-06-09 21:00:00+00:00           42.074501           27.574499   
...                        ...                 ...                 ...   
3284 2024-06-01 21:00:00+00:00           42.722000           29.022001   
3285 2024-06-02 21:00:00+00:00           42.822002           27.472000   
3286 2024-06-03 21:00:00+00:00           44.222000           28.672001   
3287 2024-06-04 21:00:00+00:00           43.622002           29.372002   
3288 2024-06-05 21:00:00+00:00           4

,date,temperature_2m_max,temperature_2m_min,temperature_2m_mean,daylight_duration,sunshine_duration
0,2015-06-05 21:00:00+00:00,43.274502,25.624498,35.437000,48978.695312,43825.601562
1,2015-06-06 21:00:00+00:00,42.974503,23.924500,34.895336,49005.664062,43703.218750
2,2015-06-07 21:00:00+00:00,42.174500,27.724499,35.234917,49030.687500,43703.613281
3,2015-06-08 21:00:00+00:00,40.824501,27.024500,34.209919,49053.746094,43704.039062
4,2015-06-09 21:00:00+00:00,42.074501,27.574499,35.197422,49075.015625,43704.527344
